In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np

In [2]:
#URL and Header
URL = "https://www.amazon.in/gp/bestsellers/computers/1375424031"
HEADERS = ({'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36','Accept-Language':'en-US,en;q=0.5'})

In [3]:
#HTTP Request
webpage = requests.get(URL, headers=HEADERS)
type(webpage.content)
print(webpage.content)

b'<!doctype html><html lang="en-in" class="a-no-js" data-19ax5a9jf="dingo"><!-- sp:feature:head-start -->\n<head><script>var aPageStart = (new Date()).getTime();</script><meta charset="utf-8"/>\n<!-- sp:end-feature:head-start -->\n<!-- sp:feature:csm:head-open-part1 -->\n\n<script type=\'text/javascript\'>var ue_t0=ue_t0||+new Date();</script>\n<!-- sp:end-feature:csm:head-open-part1 -->\n<!-- sp:feature:cs-optimization -->\n<meta http-equiv=\'x-dns-prefetch-control\' content=\'on\'>\n<link rel="dns-prefetch" href="https://images-eu.ssl-images-amazon.com">\n<link rel="dns-prefetch" href="https://m.media-amazon.com">\n<link rel="dns-prefetch" href="https://completion.amazon.com">\n<!-- sp:end-feature:cs-optimization -->\n<!-- sp:feature:csm:head-open-part2 -->\n<script type=\'text/javascript\'>\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function()

In [4]:
#Soup object containing all data
soup = BeautifulSoup(webpage.content, "html.parser")

In [5]:
# Fetch links as List of Tag Objects
links = soup.find_all("a", attrs={'class':'a-link-normal'})

In [6]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={"class": "a-offscreen"}).text

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price
# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("span",attrs={"class":'a-icon-alt'}).get_text(strip=True)
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).text

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [7]:
#Loop for extracting links from Tag objects and then product details from each link

product_links_list =[]
d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}

for link in links:
    product_links_list.append(link.get('href'))
    
for product_link in product_links_list:
    product_link = "https://www.amazon.in" + product_link 
    print(product_link)
    new_webpage = requests.get(product_link, headers=HEADERS)
    new_soup = BeautifulSoup(new_webpage.content, "html.parser")

    
    d['title'].append(get_title(new_soup))
    d['price'].append(get_price(new_soup))
    d['rating'].append(get_rating(new_soup))
    d['reviews'].append(get_review_count(new_soup))
    d['availability'].append(get_availability(new_soup))



https://www.amazon.in/Lenovo-i3-1115G4-Warranty-Platinum-81X800LCIN/dp/B0B9YHX39M/ref=zg_bs_1375424031_sccl_1/257-5056479-1060518?psc=1
https://www.amazon.in/Lenovo-i3-1115G4-Warranty-Platinum-81X800LCIN/dp/B0B9YHX39M/ref=zg_bs_1375424031_sccl_1/257-5056479-1060518?psc=1
https://www.amazon.in/product-reviews/B0B9YHX39M/ref=zg_bs_1375424031_cr_sccl_1/257-5056479-1060518
https://www.amazon.in/Lenovo-i3-1115G4-Warranty-Platinum-81X800LCIN/dp/B0B9YHX39M/ref=zg_bs_1375424031_sccl_1/257-5056479-1060518?psc=1
https://www.amazon.in/Apple-MacBook-Chip-13-inch-256GB/dp/B08N5W4NNB/ref=zg_bs_1375424031_sccl_2/257-5056479-1060518?psc=1
https://www.amazon.in/Apple-MacBook-Chip-13-inch-256GB/dp/B08N5W4NNB/ref=zg_bs_1375424031_sccl_2/257-5056479-1060518?psc=1
https://www.amazon.in/product-reviews/B08N5W4NNB/ref=zg_bs_1375424031_cr_sccl_2/257-5056479-1060518
https://www.amazon.in/Apple-MacBook-Chip-13-inch-256GB/dp/B08N5W4NNB/ref=zg_bs_1375424031_sccl_2/257-5056479-1060518?psc=1
https://www.amazon.in/A

In [8]:

import time
df = pd.DataFrame.from_dict(d)
df['title'].replace('', np.nan, inplace=True)
df = df.dropna(subset=['title'])
timestr = timestr = time.strftime("%Y%m%d-%H%M%S")
df.to_csv("amazon_data_"+timestr+".csv", header=True, index=False)